# Parameter Sensitivity Analysis
**Date:** 07.10.2024 <br>
**Name:** Silja Svendsen <br>
**Description:** Redo results and organise code for readability.

In [1]:
import pandas as pd
import sympy as sb

In [2]:
from Module_241021 import *

In [3]:
def sort_get_nanog_gata(SFP):
    """
    sorts points in order EPI, PRE
    coordinates are given as (Gata6, Nanog)
    sanity check, applied on mean sfp values.
    """
    return SFP[SFP[:,0].argsort()][:,:2]

In [4]:
# Get reference data (GN-F network, par0)
ICs = generate_ic_to_scan_deterministic_triangle(p=par0,
                                                 ic_range=[0.1, 5.0],
                                                 N_points_1D=36,
                                                 base=2,
                                                 dim=2
                                                )

GN_ss_coord, GN_ss_time = ss_coord_and_delay_time(points_ic=ICs[:,:2],
                                                  par=par0,
                                                  model=equations_NG_F
                                                 )

GN_num_sfp, GN_mean_sfp, GN_std_sfp = group_points_mean_std(points=GN_ss_coord,
                                                            radius=2
                                                           )

In [5]:
print(f"The number of initial conditions (I.C.) is {len(ICs)}")
print(f"The reference Epi and PrE states are at",
      f"\n (Gata6, Nanog)={GN_mean_sfp[0]}",
      f"\n (Gata6, Nanog)={GN_mean_sfp[1]}" 
     )

The number of initial conditions (I.C.) is 1127
The reference Epi and PrE states are at 
 (Gata6, Nanog)=[ 1.10273171 11.46535802] 
 (Gata6, Nanog)=[13.62042978  2.41356951]


In [7]:
break # remove this to run the PSA. it takes about 3hrs.

SyntaxError: 'break' outside loop (1246755309.py, line 1)

In [6]:
# Declare reference parameter set to ensure it is the correct one
par0909= {
    'basal_N':3.5,
    'basal_G':1,# 0.5/20,
    'basal_E': 1, # E parameter to optimise
    'basal_O': 1,
    'wf_G' : 14,
    'wf_N' : 14,
    'wf_E' : 12, # E parameter to optimise
    'wf_O' : 12, # doesnt change (G,N)ss if 12 or 4
    'tau_G' : 1,
    'tau_N' : 1,
    'tau_E' : 1,
    'tau_O': 1, # in Indra's thesis tauO = 3.6*tauN,G,E  # for now (03/09/24), keep tauO= 1, naive case .
    'K_GG' : 1.2,
    'K_NN' : 1.2,
    'K_GN' : 4.2,
    'K_NG' : 4.2,
    'K_FN' : 1, #1
    'K_FE' : 3, #3 # E parameter to optimise
    'K_FO' : 3, # doesnt change (G,N)ss if 1 or 3
    'K_EN' : 1.2, # E parameter to optimise
    'K_NE' : 1.2, # E parameter to optimise
    'K_NEG' : 1.2,
    'K_EG' : 1.2, # E parameter to optimise
    'K_OG' :1.2,
    'K_ON' :1.2,
    'K_NO' :1.2,
    'K_OE':1.2,
    'K_EO':1.2,
    'h' : 4, # could possibly be lowered??
    'FGF' : 0.85,
}

# reference
ref_par = par0909.copy()
ref_sfp_coord = GN_mean_sfp.copy()

# filename
network = ("GN", "GNE", "GNEO1", "GNEO2")
parameter = ("alphaGN","alphaEO",
             "KaaNG",
             "KmiNG","KmiEN",
             "KNEG", "KEG", "KOG",
             "KmaON","KmaOE"
            )
parameter_change = [0.20, 0.25, 0.50, 0.75, 1.0,
                    1.25, 2.0, 3.0, 4.0, 5.0
                   ]
date = "241021"

# (nested) for loops
keys_varied = ("wf_G", "wf_E",
               "K_GG",
               "K_GN", "K_EN",
               "K_NEG", "K_EG", "K_OG",
               "K_ON", "K_OE"
              )
model = [equations_NG_F,
         equations_NGE_and_F,
         equations_NGEO_and_F,
         equations_NGEOF_and_maOE
        ]
dimension = [2, 3, 4, 4]

# get measurements
for network_i, model_i, dim_i in zip(network, model, dimension):
    print(f"Network: {network_i}")
    
    for parameter_i, key_i in zip(parameter, keys_varied):
        print(f"parameter i: {key_i}, check ref sfp: {ref_sfp_coord}")
        
        for change in parameter_change:
            
            # 0. adjust the parameter of interest
            par0909[key_i]=change*ref_par[key_i]
            
            print(f"Debugging purposes: key: {par0909[key_i]}, rel change: {change}")
            
            # equal regulation
            par0909["wf_N"]=par0909["wf_G"]
            par0909["wf_O"]=par0909["wf_E"]
            par0909["K_NN"]=par0909["K_GG"]
            par0909["K_NG"]=par0909["K_GN"]
            par0909["K_NE"]=par0909["K_EN"]
            par0909["K_NO"]=par0909["K_ON"]
            par0909["K_EO"]=par0909["K_OE"]
            
            # 1. produce initial conditions
            ICs = generate_ic_to_scan_deterministic_triangle(
                p=par0909,
                ic_range=[0.1, 5.0],
                N_points_1D=36,
                base=2,
                dim=4
            )
            
            # 2. calculate steady state and delay times
            ss_coord, ss_time = ss_coord_and_delay_time(
                points_ic=ICs[:,:dim_i],
                par=par0909,
                model=model_i
            )
            
            # 3. calculate num sfp and <sfp coord (gata, nanog)>
            num_sfp, mean_sfp, std_sfp = group_points_mean_std(
                points=ss_coord,
                radius=2
            )
            
            # 4. calculate velocity using only gata and nanog
            distance = euclidean_distance(
                start_arr=ICs[:,:2],
                final_arr=ss_coord[:,:2]
            )
            velocity = distance/ss_time
            
            # 5. check condition 1 and 2
            if num_sfp == 2:
                
                # condition 1
                condition1 = True
                
                # sanity check: save mean_sfp gata and nanog coords only
                GN = sort_get_nanog_gata(mean_sfp) # epi, pre
                
                # compare only gata and nanog coordinates
                condition2 = condition_2(
                    SFP=mean_sfp,
                    SFP_ref=ref_sfp_coord,
                    threshold=0.20
                )
            elif num_sfp != 2:
                
                # condition 1
                condition1 = False
                
                # Accommadate for not found value error
                GN = -1*np.ones((2,2))
                
                # Does not make sense to check if not bi-stable
                condition2 = None 
                
            # 6. acommadate for out of bounds error
            # Not relevant for the data
            if dim_i == 2:
                index_esrrb = 0 # None, gata6
                index_oct4 = 0 # None, gata6
            if dim_i == 3:
                index_esrrb = 2
                index_oct4 = 0 # None, gata6
            if dim_i == 4:
                index_esrrb = 2
                index_oct4 = 3

            # sample to dataframe
            data = {
                "network": network_i,
                "num variables":dim_i,
                "parameter": parameter_i,
                "parameter value":par0909[key_i],
                "rel change": change,
                "ic gata6": ICs[:,0],
                "ic nanog": ICs[:,1],
                "ic esrrb": ICs[:,2],
                "ic oct4": ICs[:,3],
                "ss gata6": ss_coord[:,0],
                "ss nanog": ss_coord[:,1],
                "ss esrrb": ss_coord[:,index_esrrb], # dim_i>=3
                "ss oct4": ss_coord[:,index_oct4], # dim_i =4
                "num sfp": num_sfp,
                "mean epi gata6":GN[0][0], # epi gata6
                "mean epi nanog":GN[0][1], # epi nanog
                "mean pre gata6":GN[1][0], # pre gata6
                "mean pre nanog":GN[1][1], # pre nanog
                "condition 1": condition1,
                "condition 2": condition2,
                "time ss": ss_time,
                "distance": distance,
                "velocity": velocity,
            }
            
            # de-bugging purposes
            for key, value in data.items():
                print(f"{key}: {np.shape(value)}") 
            
            # save data to csv file
            dataframe = pd.DataFrame(data)
            dataframe.to_csv(f"{network_i}_{parameter_i}_{change}_{len(ICs)}_{date}.csv",
                             mode="w", index=False
                            )
            
            # reset par0909
            par0909=ref_par.copy()
            print(f"Debugging: Are the parameters reset?\n {par0909}")
            
print("DONE")

Network: GN
parameter i: wf_G, check ref sfp: [[ 1.10273171 11.46535802]
 [13.62042978  2.41356951]]
Debugging purposes: key: 2.8000000000000003, rel change: 0.2
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 3.

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 3.0, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 1.05, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4:

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

...

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 

network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: (1127,)
ss gata6: (1127,)
ss nanog: (1127,)
ss esrrb: (1127,)
ss oct4: (1127,)
num sfp: ()
mean epi gata6: ()
mean epi nanog: ()
mean pre gata6: ()
mean pre nanog: ()
condition 1: ()
condition 2: ()
time ss: (1127,)
distance: (1127,)
velocity: (1127,)
Debugging: Are the parameters reset?
 {'basal_N': 3.5, 'basal_G': 1, 'basal_E': 1, 'basal_O': 1, 'wf_G': 14, 'wf_N': 14, 'wf_E': 12, 'wf_O': 12, 'tau_G': 1, 'tau_N': 1, 'tau_E': 1, 'tau_O': 1, 'K_GG': 1.2, 'K_NN': 1.2, 'K_GN': 4.2, 'K_NG': 4.2, 'K_FN': 1, 'K_FE': 3, 'K_FO': 3, 'K_EN': 1.2, 'K_NE': 1.2, 'K_NEG': 1.2, 'K_EG': 1.2, 'K_OG': 1.2, 'K_ON': 1.2, 'K_NO': 1.2, 'K_OE': 1.2, 'K_EO': 1.2, 'h': 4, 'FGF': 0.85}
Debugging purposes: key: 0.3, rel change: 0.25
network: ()
num variables: ()
parameter: ()
parameter value: ()
rel change: ()
ic gata6: (1127,)
ic nanog: (1127,)
ic esrrb: (1127,)
ic oct4: 